In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, array_to_img, img_to_array
from skimage import filters
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import requests
from pathlib import Path
import os

from tqdm import tqdm

# coba
from PIL.ImageOps import invert

In [2]:
PATH = "../../Dataset/Dataset RSUA"
PATH_RESULT = "../../Dataset/Hasil Annotation Dataset RSUA/"

In [3]:
def lineSegmentationCXR(image_path):
    # print(image_path)
    if "Putih" in image_path:
        image = load_img(image_path, color_mode='grayscale', target_size=(256, 256))
        image = invert(image)
    else:
        image = load_img(image_path, color_mode='grayscale', target_size=(256, 256))

    # Image Array
    image_arr = img_to_array(image).astype('float32') / 255.0
    image_arr_reshape = image_arr[np.newaxis, ...] # Reshape menjadi (1, width, height, channel) agar bisa predict

    # Prediksi Model UNet
    UNet_trained_model = tf.keras.models.load_model('UNet_Model_Trained.h5', compile=False)
    mask_arr = UNet_trained_model.predict(image_arr_reshape, verbose=0)

    # Create Line Segmentation
    line_mask_arr = filters.sobel(mask_arr[0])

    # Convert Image to uint8
    image_arr_uint8 = (image_arr*255).astype(np.uint8)
    line_mask_arr_uint8 = (line_mask_arr*255).astype(np.uint8)
    
    # Convert Image to PIL
    image_pil = Image.fromarray(image_arr_uint8[:, :, 0])
    line_mask_pil = Image.fromarray(line_mask_arr_uint8[:, :, 0])

    # Convert image to RGBA
    line_mask_pil_rgba = line_mask_pil.convert("RGBA")
    image_pil_rgba = image_pil.convert("RGBA")

    # Convert image to tuple data
    line_mask_pil_data = line_mask_pil_rgba.getdata()
    image_pil_data = image_pil_rgba.getdata()

    image_result = []
    for mask_line_channels, image_channels in zip(line_mask_pil_data, image_pil_data):
        if mask_line_channels != (0, 0, 0, 255):
            image_channels = list(image_channels)
            image_channels[0] = int((image_channels[0] + 180) / 2)
            image_channels[1] = int((image_channels[1] + 0) / 2)
            image_channels[2] = int((image_channels[2] + 0) / 2)
            # image_channels[3] = 120
            image_channels = tuple(image_channels)
            image_channels = image_channels
            image_result.append(image_channels)
        else:
            image_result.append(image_channels)
    image_pil_rgba.putdata(image_result)

    IMAGE = image
    MASK = array_to_img(mask_arr[0])
    LINE_MASK = image_pil_rgba
    
    new_im_merge = Image.new('RGB', (256*3, 256))
    new_im_merge.paste(IMAGE, (0, 0))
    new_im_merge.paste(LINE_MASK, (256, 0))
    new_im_merge.paste(MASK, (256*2, 0))

    return IMAGE, MASK, LINE_MASK, new_im_merge


In [4]:
def getAllFilespath(PATH):
    covid_filespath = []
    non_covid_filespath = []
    non_covid_pneumonia_filespath = []
    image_label = ['Covid', 'Non_Covid', 'Non_Covid_Pneumonia']
    i = 0
    for root, dirs, files in os.walk(PATH):
        if Path(root).name in image_label:
            i += 1
            root_images = root  # Get name root folder until file
            image_filenames = os.listdir(root_images)  # Get All image filenames
            # Get 1 by 1 filepath and append to array
            for image_filename in image_filenames:
                path_file = os.path.join(root_images, image_filename)
                if Path(root).name == 'Covid':
                    covid_filespath.append(path_file)
                elif Path(root).name == 'Non_Covid':
                    non_covid_filespath.append(path_file)
                elif Path(root).name == 'Non_Covid_Pneumonia':
                    non_covid_pneumonia_filespath.append(path_file)

    return covid_filespath, non_covid_filespath, non_covid_pneumonia_filespath


covid_filespath, non_covid_filespath, non_covid_pneumonia_filespath = getAllFilespath(PATH)

In [5]:
def run(images_path, kind):
    for i in tqdm(range(len(images_path))):
        image, mask, line_mask, merge_image = lineSegmentationCXR(images_path[i])
        _ = image.save(PATH_RESULT + "{}/images/".format(kind) + "{}_{}".format(kind, i+1)+".bmp") # image
        _ = mask.save(PATH_RESULT + "{}/masks/".format(kind) + "{}_{}".format(kind, i+1)+".bmp")  # mask
        _ = line_mask.save(PATH_RESULT + "{}/line_masks/".format(kind) + "{}_{}".format(kind, i+1)+".bmp")  # line_mask
        _ = merge_image.save(PATH_RESULT + "{}/merge_images/".format(kind) + "{}_{}".format(kind, i+1)+".bmp")  # merge_image


In [6]:
run(covid_filespath, "Covid")

  0%|          | 0/273 [00:00<?, ?it/s]2022-11-16 11:26:33.893595: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-16 11:26:33.893699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-11-16 11:26:34.572726: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-16 11:26:34.691514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
  1%|▏         | 4/273 [00:03<04:00,  1.12it/s]

2022-11-16 11:26:38.213241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
  2%|▏         | 5/273 [00:04<03:44,  1.19it/s]2022-11-16 11:26:39.444929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


100%|██████████| 273/273 [04:34<00:00,  1.00s/it]


In [7]:
run(non_covid_filespath, "Non_Covid")


100%|██████████| 39/39 [00:47<00:00,  1.21s/it]


In [8]:
run(non_covid_pneumonia_filespath, "Non_Covid_Pneumonia")


100%|██████████| 59/59 [01:13<00:00,  1.25s/it]
